## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [113]:
import pandas as pd

In [114]:
film_data = pd.read_csv('ml-latest-small/ratings.csv')
film_data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [115]:
# у каждого фильма есть много оценок, поэтому, сначала посчитаем среднюю оценку для каждого
film_rating = film_data.groupby('movieId').mean()[['rating']].reset_index()
film_rating.head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857


In [29]:
def group_by_class(row):
    if row.rating <= 2:
        return 'Low'
    elif 2 <= row.rating <= 4:
        return 'Middle'
    elif row.rating == 4.5 or row.rating == 5:
        return 'High'

film_rating['class'] = film_rating.apply(group_by_class, axis=1)
film_rating.sort_values('rating').head()

,movieId,rating,class
7592,80350,0.5,Low
4340,5850,0.5,Low
6524,48591,0.5,Low
6522,48522,0.5,Low
4345,5864,0.5,Low


## Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [2]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [3]:
key_data = pd.read_csv('keywords.csv')
key_data.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [23]:
# этот вариант будет работать даже если в geo_data будут добавляться другие города и регионы

def find_geo(row):
    finding_region = 'undefined'
    
    for region in geo_data.items():
        if any(city in row.keyword for city in region[1]):
            finding_region = region[0]
    
    return finding_region

In [15]:
# простой вариант в лоб

def find_geo_light(row):
    if any(city in row.keyword for city in geo_data['Центр']):
        return 'Центр'
    elif any(city in row.keyword for city in geo_data['Северо-Запад']):
        return 'Северо-Запад'
    elif any(city in row.keyword for city in geo_data['Дальний Восток']):
        return 'Дальний Восток'
    else:
        return 'undefined'

In [22]:
key_data['region'] = key_data.apply(find_geo, axis=1)
key_data.head()

# проверка провильности
# key_data[key_data['keyword'].str.contains('владивосток')].head()

,keyword,shows,region
1063,фарпост владивосток,176951,Дальний Восток
3206,дром владивосток,73084,Дальний Восток
3275,100 сп владивосток,76418,Дальний Восток
5858,владивосток,43038,Дальний Восток
16544,погода во владивостоке,18262,Дальний Восток


## Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [104]:
films_data = pd.read_csv('ml-latest-small/movies.csv')
films_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [140]:
# делаем список годов, сразу в str
years = []

for i in  range(1950, 2011):
    years.append(str(i))

In [156]:
# год в названии будем искать простой регуляркой
import re

def production_year(row):
    year = ''
    
    # проверяем, если ли вообще год, чтобы потом не нарваться на ошибку
    if re.findall("[0-9]{4}", row.title):
        year = re.findall("[0-9]{4}", row.title)[0]
    
    if year in years:
        return year
    else:
        return '1900'

In [157]:
films_data['year'] = films_data.apply(production_year, axis=1)
films_data.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [158]:
# добавляем средний рейтинг каждого фильма из первого задания
# не у каждого фильма есть рейтинг, поэтому склеиваем по movieId

films_years_ratings = films_data.join(film_rating.set_index('movieId'), on='movieId')
films_years_ratings.head()

,movieId,title,genres,year,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.384615
4,5,Father of the Bride Part II (1995),Comedy,1995,3.267857


In [164]:
# выводим ТОП10 годов по рейтингу

films_years_ratings.groupby('year').rating.mean().reset_index().sort_values('rating', ascending=False).head(10)

,year,rating
2,1951,3.906653
3,1952,3.802318
13,1962,3.772962
4,1953,3.738878
8,1957,3.693322
24,1973,3.688212
22,1971,3.655363
20,1969,3.604509
23,1972,3.585295
9,1958,3.571956
